In [51]:
import os
import pandas as pd
import requests
from decouple import AutoConfig
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

In [52]:
config = AutoConfig(search_path="../.env")

GROQ_API_KEY = config("GROQ_API_KEY")
# LANGCHAIN_TRACING_V2 = config("LANGCHAIN_TRACING_V2")
# LANGCHAIN_ENDPOINT = config("LANGCHAIN_ENDPOINT")
# LANGCHAIN_PROJECT = config("LANGCHAIN_PROJECT")
# lANGCHAIN_API_KEY = config("LANGCHAIN_API_KEY")

In [53]:
job_url = input("Paste the URL here: ")
print(f"{job_url=}")

job_url='https://www.amazon.jobs/en/jobs/2831807/sde'


In [54]:
web_data = WebBaseLoader(web_path=job_url).load()
llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY, temperature=0.5)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
json_parser = JsonOutputParser()
str_parser = StrOutputParser()

In [55]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [56]:
scrape_prompt = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        THe Scraped text is from the career page of a website.
        your job is to extract the job postings and return them in JSON format containing
        the following keys: `role`, `experience`, `skills` and `description`.
        Return only the valid JSON.
        ### VALID JSON (NO PREAMBLE):
    """
)

email_prompt = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    ### INSTRUCTION:
    You are Madhu, a business development executive at Mocha. 
    Mocha is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing 
    the capability of Mocha in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Mocha's portfolio: {link_list}
    Remember you are Madhu, BDE at Mocha. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

In [57]:
scrape_chain = scrape_prompt | llm | json_parser
email_chain = email_prompt | llm | str_parser

In [58]:
page_data = web_data.pop().page_content
print(scrape_prompt.format(**{"page_data":page_data}))


        ### SCRAPED TEXT FROM WEBSITE:
        SDE - Job ID: 2831807 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×SDEJob ID: 2831807 | ADCI - KarnatakaApply nowDESCRIPTIONABOUT THIS ROLEAndroid Software Development Engineers build user interfaces, expand and improve deployment, testing and work with new technologies powered by Amazon.This is 12 months contract role with Amazon , Bangalore.ABOUT THE TEAMWe own the end to end shopping experience for the mobile app which includes web, iOS & Android, content security & digital rights management at a massive scale.ABOUT YOUYou’re a problem solver. A fast-paced environment fuels your creativity – not to mention your team who inspire you.Through plenty of practice you’ve mastered how to translate even the most complex ideas in a

In [59]:
response = scrape_chain.invoke(input={"page_data": page_data})
print(response)

{'role': 'Android Software Development Engineer', 'experience': '2+ years of experience in designing and developing Android Applications using Kotlin, Java, the Android SDK, and common Android tools and platforms like Compose, Jetpack Navigation', 'skills': ['Kotlin', 'Java', 'Android SDK', 'Android', 'Compose', 'Jetpack Navigation', 'React', 'CSS', 'VueJS', 'HTML5', 'Object-oriented development', 'Multithreading', 'Data structures'], 'description': 'Android Software Development Engineers build user interfaces, expand and improve deployment, testing and work with new technologies powered by Amazon. This is a 12-month contract role with Amazon, Bangalore.'}


In [60]:
df = pd.read_csv("./my_portfolio.csv")
display(df)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [61]:
csv_documents = []
for _, row in df.iterrows():
    csv_documents.append(Document(page_content=row["Techstack"], metadata={"source": row["Links"]}))

In [62]:
vector_store = Chroma(collection_name="portfolio_collection", embedding_function=embeddings)
vector_store.add_documents(csv_documents)

['d4ab3301-be73-425c-bc0d-c2a9b4c00656',
 '571824cc-5c96-4b05-9b95-8b91493a9d21',
 '6aa7ab9b-6d1a-4741-9e2d-e4c22b55ff18',
 'ea3a9c26-e9f8-4758-a3aa-68bdd8ae939c',
 '7d959447-4a38-4b70-b332-9e55ada405f0',
 '163d332b-63aa-403e-83b2-c0f0d96c0eff',
 '0244e0dd-9811-461b-80b3-0006ed916155',
 'c601bf0c-9453-401f-8c9d-8e81009887f9',
 '536f8e0c-113e-4c5a-bda9-76872030c5b4',
 'fd0c1387-ad55-449b-8688-b0290c5a6fbf',
 'bd3c7e7b-62f2-4ec4-9f73-187fa2fa2e60',
 '8d733765-6f86-4983-861b-df3b569a891e',
 '36334e88-b15a-4571-994a-e30956b6bbde',
 'f3f05485-19a0-4e75-afd0-1802c396b6c2',
 'a806075e-0b5e-403c-a19d-6cab0611b644',
 'a726a50d-a46c-4982-8e97-aa752674460f',
 '1cd3a0ad-20b1-4134-89b6-a9e8a7886e68',
 'f42002f1-8135-4de8-9089-1fb4784ae6c6',
 '5fd5b11a-ac80-4940-b302-33d01c6bbe74',
 '161383e0-c35f-417d-9644-5d133b3bbbc1']

In [63]:
response['skills']

['Kotlin',
 'Java',
 'Android SDK',
 'Android',
 'Compose',
 'Jetpack Navigation',
 'React',
 'CSS',
 'VueJS',
 'HTML5',
 'Object-oriented development',
 'Multithreading',
 'Data structures']

In [64]:
relevant_portfolios = []
for skill in response.get('skills'):
    relevant_portfolios.append(vector_store.similarity_search(skill, k=1)[0].metadata['source'])
relevant_portfolios

['https://example.com/kotlin-android-portfolio',
 'https://example.com/java-portfolio',
 'https://example.com/android-tv-portfolio',
 'https://example.com/android-tv-portfolio',
 'https://example.com/full-stack-js-portfolio',
 'https://example.com/ios-ar-portfolio',
 'https://example.com/react-portfolio',
 'https://example.com/wordpress-portfolio',
 'https://example.com/vue-portfolio',
 'https://example.com/full-stack-js-portfolio',
 'https://example.com/java-portfolio',
 'https://example.com/ml-python-portfolio',
 'https://example.com/ios-portfolio']

In [65]:
print(email_prompt.format(**{"job_description":str(response), "link_list":set(relevant_portfolios)}))


    ### JOB DESCRIPTION:
    {'role': 'Android Software Development Engineer', 'experience': '2+ years of experience in designing and developing Android Applications using Kotlin, Java, the Android SDK, and common Android tools and platforms like Compose, Jetpack Navigation', 'skills': ['Kotlin', 'Java', 'Android SDK', 'Android', 'Compose', 'Jetpack Navigation', 'React', 'CSS', 'VueJS', 'HTML5', 'Object-oriented development', 'Multithreading', 'Data structures'], 'description': 'Android Software Development Engineers build user interfaces, expand and improve deployment, testing and work with new technologies powered by Amazon. This is a 12-month contract role with Amazon, Bangalore.'}
    ### INSTRUCTION:
    You are Madhu, a business development executive at Mocha. 
    Mocha is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with 

In [66]:
response = email_chain.invoke({"job_description":str(response), "link_list":set(relevant_portfolios)})
print(response)

Subject: Expert Android Software Development Services for Amazon Project

Dear Hiring Manager,

I came across the job description for an Android Software Development Engineer at Amazon, Bangalore, and I'm excited to introduce Mocha, a leading AI & Software Consulting company. With our expertise in designing and developing Android Applications using Kotlin, Java, and the Android SDK, I believe we can fulfill your requirements.

At Mocha, we have a proven track record of delivering high-quality Android solutions. Our team of experts has extensive experience in building user interfaces, expanding and improving deployment, testing, and working with new technologies. We're well-versed in Compose, Jetpack Navigation, and other common Android tools and platforms. Our proficiency in object-oriented development, multithreading, and data structures ensures that our solutions are scalable, efficient, and meet the highest standards.

I'd like to highlight some of our notable projects that demonstr

In [67]:
api_key = GROQ_API_KEY
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
for model in response.json()['data']:
    print(model['id'], '--', model['context_window'])

llama-3.1-8b-instant -- 131072
llama-guard-3-8b -- 8192
llama-3.1-70b-versatile -- 32768
llama3-70b-8192 -- 8192
llama3-groq-8b-8192-tool-use-preview -- 8192
llama-3.3-70b-specdec -- 8192
llama-3.2-90b-vision-preview -- 8192
mixtral-8x7b-32768 -- 32768
distil-whisper-large-v3-en -- 448
llama-3.2-1b-preview -- 8192
llama3-groq-70b-8192-tool-use-preview -- 8192
llama3-8b-8192 -- 8192
llama-3.2-3b-preview -- 8192
gemma2-9b-it -- 8192
llama-3.2-11b-vision-preview -- 8192
whisper-large-v3 -- 448
whisper-large-v3-turbo -- 448
llama-3.3-70b-versatile -- 32768
